In [ ]:
import numpy as np

class GreyWolfOptimizer:
    def __init__(self, function, bounds, num_wolves=30, max_iter=100):
        self.function = function
        self.bounds = bounds
        self.num_wolves = num_wolves
        self.max_iter = max_iter
        self.dim = len(bounds)
        self.wolves_pos = np.random.rand(num_wolves, self.dim)
        # Scale positions to fit within the bounds
        for i in range(self.dim):
            self.wolves_pos[:, i] = self.wolves_pos[:, i] * (bounds[i][1] - bounds[i][0]) + bounds[i][0]
        self.alpha_pos = np.zeros(self.dim)
        self.alpha_score = float("inf")
        self.beta_pos = np.zeros(self.dim)
        self.beta_score = float("inf")
        self.delta_pos = np.zeros(self.dim)
        self.delta_score = float("inf")

    def evaluate_fitness(self, position):
        return self.function(position)

    def update_position(self, a, alpha_pos, beta_pos, delta_pos, wolves_pos):
        # Generate random values for exploration
        r1, r2, r3 = np.random.rand(3, self.num_wolves, self.dim)

        # Coefficients for position update
        A = 2 * a * r1 - a
        C = 2 * r2

        # Distances from each wolf to the alpha, beta, and delta wolves
        D_alpha = np.abs(C[0] * alpha_pos - wolves_pos)
        D_beta = np.abs(C[1] * beta_pos - wolves_pos)
        D_delta = np.abs(C[2] * delta_pos - wolves_pos)

        # New positions based on the social hierarchy
        X1 = alpha_pos - A[0] * D_alpha
        X2 = beta_pos - A[1] * D_beta
        X3 = delta_pos - A[2] * D_delta

        # Update positions of all wolves
        return (X1 + X2 + X3) / 3

    def optimize(self):
        for t in range(self.max_iter):
            a = 2 - t * (2 / self.max_iter)  # Coefficient decreases over time
            for i in range(self.num_wolves):
                fitness = self.evaluate_fitness(self.wolves_pos[i])
                # Update alpha, beta, and delta wolves based on fitness
                if fitness < self.alpha_score:
                    self.alpha_score, self.alpha_pos = fitness, self.wolves_pos[i]
                elif fitness < self.beta_score:
                    self.beta_score, self.beta_pos = fitness, self.wolves_pos[i]
                elif fitness < self.delta_score:
                    self.delta_score, self.delta_pos = fitness, self.wolves_pos[i]
            # Update the positions of wolves
            self.wolves_pos = self.update_position(a, self.alpha_pos, self.beta_pos, self.delta_pos, self.wolves_pos)
        return self.alpha_pos, self.alpha_score

# Example function to optimize (Sphere Function)
def sphere_function(x):
    return np.sum(x**2)

# Set bounds and run GWO optimizer
bounds = [(-5, 5), (-5, 5)]
gwo = GreyWolfOptimizer(function=sphere_function, bounds=bounds, num_wolves=30, max_iter=100)
best_position, best_score = gwo.optimize()

print("Best Position:", best_position)
print("Best Score:", best_score)


Best Position: [0.00157709 0.00146409]
Best Score: 4.630776977407126e-06
